In [1]:
import pandas as pd
df = pd.DataFrame(columns = ["Autistic", "Drew on house", "stroke duration", "colors used", "total gestures", "total time"])

In [4]:
import tkinter as tk
import time

class DrawingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Drawing App")

        self.canvas = tk.Canvas(root, width=500, height=500, bg="white")
        self.canvas.pack()
        

        self.colors = ["red", "green", "blue", "yellow", "black", "orange", "purple", "pink", "brown", "cyan"]
        self.current_color = self.colors[0]

        self.color_frame = tk.Frame(root)
        self.color_frame.pack(pady=10)

        for color in self.colors:
            color_button = tk.Button(self.color_frame, bg=color, width=4, height=2, command=lambda c=color: self.change_color(c))
            color_button.pack(side="left", padx=5)

        self.user_drew_on_house = False  # Track if the user added to the house drawing
        self.start_time = time.time()
        self.gesture_start_time = None
        self.gesture_speeds = []
        self.gesture_durations = []
        self.canvas.create_text(250, 40, text="Draw a farm, starting with trees. \nThe CPU will draw extra details on your picture. \nFeel free to draw on the CPU details", font=("Helvetica", 15), fill="black")

        

        self.root.after(10000, self.draw_house)  # Schedule drawing house after 10 seconds

        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.end_paint)
        self.canvas.bind("<Motion>", self.track_speed)

        self.house_coordinates = {"x1": 20, "y1": 350, "x2": 120, "y2": 450}

        self.metrics = {
            "num_colors_used": 0,
            "num_gestures": 0,
            "avg_duration": 0,
            "avg_speed": 0,
            "total_time": 0
        }

        print("Start drawing something!")

    def change_color(self, color):
        self.current_color = color
        self.metrics["num_colors_used"] += 1

    def paint(self, event):
        x, y = event.x, event.y
        oval_size = 5  # Adjust the oval size as needed for smooth drawing
        x1, y1 = (x - oval_size), (y - oval_size)
        x2, y2 = (x + oval_size), (y + oval_size)
        self.canvas.create_oval(x1, y1, x2, y2, fill=self.current_color, width=0)  # Increase oval size and remove outline

        if self.is_point_on_house(x, y):
            self.user_drew_on_house = True

        if self.gesture_start_time is None:
            self.gesture_start_time = time.time()

    def end_paint(self, event):
        if self.gesture_start_time is not None:
            end_time = time.time()
            stroke_duration = end_time - self.gesture_start_time
            self.gesture_durations.append(stroke_duration)
            self.gesture_start_time = None
            self.metrics["num_gestures"] += 1

    def track_speed(self, event):
        if self.gesture_start_time is not None:
            current_time = time.time()
            speed = 1 / (current_time - self.gesture_start_time)
            self.gesture_speeds.append(speed)
            self.gesture_start_time = current_time

    def draw_house(self):
        # Drawing a basic house in the corner
        self.canvas.create_rectangle(
            self.house_coordinates["x1"], self.house_coordinates["y1"],
            self.house_coordinates["x2"], self.house_coordinates["y2"], fill="brown"
        )  # Base of the house
        self.canvas.create_polygon(
            self.house_coordinates["x1"], self.house_coordinates["y1"],
            (self.house_coordinates["x1"] + self.house_coordinates["x2"]) / 2, 300,
            self.house_coordinates["x2"], self.house_coordinates["y1"], fill="red"
        )  # Roof
        self.canvas.create_rectangle(40, 380, 60, 420, fill="blue")  # Window
        self.canvas.create_rectangle(80, 380, 100, 420, fill="blue")  # Window

        

        self.metrics["avg_duration"] = sum(self.gesture_durations) / len(self.gesture_durations) if self.gesture_durations else 0
        self.metrics["avg_speed"] = sum(self.gesture_speeds) / len(self.gesture_speeds) if self.gesture_speeds else 0
        self.metrics["total_time"] = time.time() - self.start_time

        

    def is_point_on_house(self, x, y):
        # Check if the point is on the house drawing
        return (
            self.house_coordinates["x1"]-100 <= x <= self.house_coordinates["x2"]+100
            and self.house_coordinates["y1"]-100 <= y <= self.house_coordinates["y2"]+100
        )

    def run(self):
        self.root.mainloop()
        print("Metrics:")
        print("Number of Colors Used:", self.metrics["num_colors_used"])
        print("Number of Gestures:", self.metrics["num_gestures"])
        print("Average Duration of Gesture:", self.metrics["avg_duration"])
        print("Average Speed of Gesture:", self.metrics["avg_speed"])
        print("Total Time Spent in the Application:", self.metrics["total_time"])
        print("User added to the house drawing:", self.user_drew_on_house)
        df.loc[len(df)] = [False, self.user_drew_on_house, self.metrics["avg_duration"], self.metrics["num_colors_used"], self.metrics["num_gestures"], self.metrics["total_time"]]


if __name__ == "__main__":
    root = tk.Tk()
    app = DrawingApp(root)
    app.run()
    df.head(100)


Start drawing something!
Metrics:
Number of Colors Used: 3
Number of Gestures: 6
Average Duration of Gesture: 0.8051636219024658
Average Speed of Gesture: 0
Total Time Spent in the Application: 10.00497031211853
User added to the house drawing: True


## LOGIT Model to predict the probability of ASU

Take P value from the model, if less than 40%, then direct user to more information on other disorders that area commonly misdiagnosed as ASU. If P value is between 40% and 70%, probe prewritten questions to best location (location, patient age, prior diagnosis, speech/communication issues?). If P value is greater than 70%, ask same probing question and schedule for an appointment but set the patient as a high urgency patient.